In [ ]:
!pip install tensorflow opencv-contrib-python youtube-dl moviepy pydot
!pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

In [ ]:
# Import the required libraries.
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [ ]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [ ]:
!apt install unzip

In [ ]:
# Create a Matplotlib figure and specify the size of the figure.
plt.figure(figsize = (20, 20))

# Get the names of all classes/categories in UCF50.
all_classes_names = os.listdir('/kaggle/input/ucf50/UCF50')

# Generate a list of 20 random values. The values will be between 0-50,
# where 50 is the total number of class in the dataset.
random_range = random.sample(range(len(all_classes_names)), 20)

# Iterating through all the generated random values.
for counter, random_index in enumerate(random_range, 1):

    # Retrieve a Class Name using the Random Index.
    selected_class_Name = all_classes_names[random_index]

    # Retrieve the list of all the video files present in the randomly selected Class Directory.
    video_files_names_list = os.listdir(f'/kaggle/input/ucf50/UCF50/{selected_class_Name}')

    # Randomly select a video file from the list retrieved from the randomly selected Class Directory.
    selected_video_file_name = random.choice(video_files_names_list)

    # Initialize a VideoCapture object to read from the video File.
    video_reader = cv2.VideoCapture(f'/kaggle/input/ucf50/UCF50/{selected_class_Name}/{selected_video_file_name}')

    # Read the first frame of the video file.
    _, bgr_frame = video_reader.read()

    # Release the VideoCapture object.
    video_reader.release()

    # Convert the frame from BGR into RGB format.
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

    # Write the class name on the video frame.
    cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame.
    plt.subplot(5, 4, counter);plt.imshow(rgb_frame);plt.axis('off')

In [ ]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 8

# Specify the directory containing the UCF50 dataset.
DATASET_DIR = "/kaggle/input/ucf50/UCF50"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = os.listdir('/kaggle/input/ucf50/UCF50')

In [ ]:
print(CLASSES_LIST)

In [ ]:
NUM_CLASSES = len(CLASSES_LIST)
print(NUM_CLASSES)

In [ ]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.
    frames_list = []

    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video.
        success, frame = video_reader.read()

        # Check if Video frame is not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)

    # Release the VideoCapture object.
    video_reader.release()

    # Return the frames list.
    return frames_list

In [ ]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []

    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):

        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

        # Iterate through all the files present in the files list.
        for file_name in files_list:

            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)

    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [ ]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()
np.save('saved_features',features)
np.save('saved_labels',labels)
np.save('saved_video_files_paths',video_files_paths)

In [ ]:
features = np.load('saved_features.npy')
labels = np.load('saved_labels.npy')
video_files_paths = np.load('saved_video_files_paths.npy')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
# Split the Data into Train ( 75% ) and Test Set ( 25% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.25, shuffle = True,
                                                                            random_state = seed_constant)

np.save('saved_features_train',features_train)
np.save('saved_features_test',features_test)
np.save('saved_labels_train',labels_train)
np.save('saved_labels_test',labels_test)

In [ ]:
features_train = np.load('/kaggle/input/features/saved_features_train.npy')
features_test = np.load('/kaggle/input/features/saved_features_test.npy')
labels_train = np.load('/kaggle/input/features/saved_labels_train.npy')
labels_test = np.load('/kaggle/input/features/saved_labels_test.npy')

In [ ]:
features_trains.shape

In [ ]:
labels_train.shape

In [ ]:
features_test.shape

In [ ]:
labels_test.shape

In [ ]:
pip install tensorflow-addons

In [ ]:
pip install keras-tcn

In [ ]:
!pip install tensorflow-addons==0.16.1


In [ ]:
import tensorflow_addons as tfa

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, GlobalAveragePooling3D, Dense, Flatten
#from tcn import TCN
#from tensorflow_addons.layers import TimeDistributedNormalization

In [ ]:

# Define the input shape (adjust this according to your data)
input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)  # Replace with actual values

# Create the input layer
input_tensor = Input(shape=input_shape)

# Define the I3D pathway
conv1 = Conv3D(64, (7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(input_tensor)
maxpool1 = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same')(conv1)

conv2 = Conv3D(64, (1, 1, 1), padding='same', activation='relu')(maxpool1)
conv3 = Conv3D(192, (3, 3, 3), padding='same', activation='relu')(conv2)
maxpool2 = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same')(conv3)

# I3D pathway's Global Average Pooling
i3d_gap = GlobalAveragePooling3D()(maxpool2)

# Define the TCN pathway
tcn_conv1 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(input_tensor)
tcn_conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(tcn_conv1)
pooling = MaxPooling3D((2, 2, 2))(tcn_conv2)

flattened = Flatten()(pooling)
fc1 = Dense(128, activation='relu')(flattened)
output_tcn = Dense(NUM_CLASSES, activation='softmax')(fc1)
#tcn_gap = GlobalAveragePooling3D()(output_tcn)

# Combine features from both pathways
combined_features = tf.keras.layers.concatenate([i3d_gap, output_tcn])

# Fully connected layers
fc2 = Dense(512, activation='relu')(combined_features)
output = Dense(NUM_CLASSES, activation='softmax')(fc2)  # Replace NUM_CLASSES with your number of activity classes

# Create the model
model = tf.keras.Model(inputs=input_tensor, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(features_trains, labels_train, epochs=20)

In [ ]:
model.save('1st_model.keras')

In [ ]:
def Inception3D(input_tensor):
    # Define the Inception module
    # You can customize the filters and kernel sizes based on your needs
    conv1x1 = Conv3D(64, (1, 1, 1), padding='same', activation='relu')(input_tensor)
    conv3x3_reduce = Conv3D(96, (1, 1, 1), padding='same', activation='relu')(input_tensor)
    conv3x3 = Conv3D(128, (3, 3, 3), padding='same', activation='relu')(conv3x3_reduce)
    conv5x5_reduce = Conv3D(16, (1, 1, 1), padding='same', activation='relu')(input_tensor)
    conv5x5 = Conv3D(32, (5, 5, 5), padding='same', activation='relu')(conv5x5_reduce)
    maxpool = MaxPooling3D((3, 3, 3), strides=(1, 1, 1), padding='same')(input_tensor)
    conv1x1_proj = Conv3D(32, (1, 1, 1), padding='same', activation='relu')(maxpool)
    inception_output = tf.keras.layers.concatenate([conv1x1, conv3x3, conv5x5, conv1x1_proj], axis=-1)
    return inception_output

In [ ]:
input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)

input_tensor = Input(shape=input_shape)

#I3D Pathway

I3D_conv1 = Conv3D(64, (7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(input_tensor)
I3D_maxpool1 = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same')(I3D_conv1)

I3D_conv2 = Conv3D(64, (1, 1, 1), padding='same', activation='relu')(I3D_maxpool1)
I3D_conv3 = Conv3D(192, (3, 3, 3), padding='same', activation='relu')(I3D_conv2)
I3D_maxpool2 = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same')(I3D_conv3)

I3D_inception3a = Inception3D(I3D_maxpool2)
I3D_inception3b = Inception3D(I3D_inception3a)
I3D_gap = GlobalAveragePooling3D()(I3D_inception3b)

#TCN Pathway

tcn_conv1 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(input_tensor)
tcn_conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(tcn_conv1)
tcn_pooling = MaxPooling3D((2, 2, 2))(tcn_conv2)

tcn_flattened = Flatten()(tcn_pooling)
#tcn_fc = Dense(128, activation='relu')(tcn_flattened)
tcn_gap = GlobalAveragePooling3D()(tcn_pooling)

#Concatenated Features
combined_features = tf.keras.layers.concatenate([I3D_gap, tcn_gap])

#Fully connected layer

fc1 = Dense(512, activation='relu')(combined_features)
output = Dense(NUM_CLASSES, activation='softmax')(fc1)

model_com = tf.keras.Model(inputs=input_tensor, outputs=output)

model_com.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_com.summary()



In [ ]:
history = model_com.fit(features_train, labels_train, epochs=30)

In [ ]:
import matplotlib.pyplot as plt

# Plotting the loss curve
plt.figure(figsize=[6,4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
#plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

# Plotting the accuracy curve
plt.figure(figsize=[6,4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
#plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)

In [ ]:
model_com.save('I3d_TCN_model.keras')

In [ ]:
plot_model(model_com, to_file="iris_model.png", show_shapes=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv1D, BatchNormalization, Activation

In [ ]:
class TemporalConvNet(Layer):
    def __init__(self, num_filters, kernel_size, dilations):
        super(TemporalConvNet, self).__init__()
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.dilations = dilations
        #self.use_batch_norm = use_batch_norm
        self.conv_layers = []

        for dilation in dilations:
            conv = Conv1D(filters=num_filters,
                          kernel_size=kernel_size,
                          dilation_rate=dilation,
                          padding='causal',
                          activation='relu',
                          kernel_initializer='he_normal')

            self.conv_layers.append(conv)

    def call(self, inputs):
        x = inputs
        for layer in self.conv_layers:
            x = layer(x)
            #if self.use_batch_norm:
                #x = BatchNormalization()(x)
            x = Activation('relu')(x)
        return x

In [ ]:
# Define TCN parameters
num_filters = 32
kernel_size = 3
dilations = [1, 2, 4]

In [ ]:
input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)

input_tensor = Input(shape=input_shape)

#I3D Pathway

I3D_conv1 = Conv3D(64, (7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(input_tensor)
I3D_maxpool1 = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same')(I3D_conv1)

I3D_conv2 = Conv3D(64, (1, 1, 1), padding='same', activation='relu')(I3D_maxpool1)
I3D_conv3 = Conv3D(192, (3, 3, 3), padding='same', activation='relu')(I3D_conv2)
I3D_maxpool2 = MaxPooling3D((1, 3, 3), strides=(1, 2, 2), padding='same')(I3D_conv3)

I3D_inception3a = Inception3D(I3D_maxpool2)
I3D_inception3b = Inception3D(I3D_inception3a)
I3D_gap = GlobalAveragePooling3D()(I3D_inception3b)


# Create TCN layer
tcn_layer_1 = TemporalConvNet(num_filters, kernel_size, dilations)(input_tensor)
tcn_layer_1 = tf.keras.layers.BatchNormalization()(tcn_layer_1)
tcn_layer_2 = TemporalConvNet(num_filters, kernel_size, dilations)(tcn_layer_1)
tcn_layer_2 = tf.keras.layers.BatchNormalization()(tcn_layer_2)
tcn_gap = GlobalAveragePooling3D()(tcn_layer_2)

#Concatenated Features
combined_features = tf.keras.layers.concatenate([I3D_gap, tcn_gap])

#Fully connected layer

fc1 = Dense(512, activation='relu')(combined_features)
output = Dense(NUM_CLASSES, activation='softmax')(fc1)

model_com = tf.keras.Model(inputs=input_tensor, outputs=output)

model_com.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_com.summary()

In [ ]:
plot_model(model_com, to_file="iris_model.png", show_shapes=True)

In [ ]:
history = model_com.fit(features_train, labels_train, epochs=30)

In [ ]:
model_com.save('Corrected_model.keras')

In [ ]:
import matplotlib.pyplot as plt

# Plotting the loss curve
plt.figure(figsize=[6,4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
#plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

# Plotting the accuracy curve
plt.figure(figsize=[6,4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
#plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)